# Export YOLOv8n COCO → TFLite INT8 for obstacle detection
Run all cells top to bottom. Takes ~5 minutes total.

## Cell 1 — Install

In [ ]:
import subprocess, sys
subprocess.run([sys.executable,"-m","pip","install","-q","ultralytics"], check=True)
import ultralytics
print("ultralytics:", ultralytics.__version__)


## Cell 2 — Export YOLOv8n COCO to TFLite INT8
This uses the standard YOLOv8n pretrained on COCO — no training needed.
Dogs, people, vehicles etc are already built in.

In [ ]:
from ultralytics import YOLO
import os, glob

# Load standard YOLOv8n (COCO pretrained — 80 classes)
model = YOLO("yolov8n.pt")

print("Exporting YOLOv8n COCO to TFLite INT8...")
print("This takes 3-5 minutes...")

model.export(
    format = "tflite",
    imgsz  = 320,
    int8   = True,
    nms    = False,
)

# Find the exported file
tflite_files = glob.glob("**/*int8*.tflite", recursive=True)
tflite_files += [f for f in glob.glob("**/*.tflite", recursive=True)
                 if f not in tflite_files]

print("\nFiles found:")
for f in tflite_files:
    print(f"  {f}  ({os.path.getsize(f)/1024/1024:.2f} MB)")

MODEL_PATH = None
for f in tflite_files:
    if "int8" in f.lower():
        MODEL_PATH = f
        break
if not MODEL_PATH and tflite_files:
    MODEL_PATH = tflite_files[0]

print("\nUsing:", MODEL_PATH)
print("Size  :", round(os.path.getsize(MODEL_PATH)/1024/1024,2), "MB")


## Cell 3 — Verify output shape

In [ ]:
import tensorflow as tf
import numpy as np

interp = tf.lite.Interpreter(model_path=MODEL_PATH)
interp.allocate_tensors()

inp = interp.get_input_details()[0]
out = sorted(interp.get_output_details(), key=lambda x: x["index"])

print("INPUT:")
print(f"  shape={inp['shape']}  dtype={inp['dtype'].__name__}")
print("\nOUTPUT:")
for d in out:
    print(f"  [{d['index']}] shape={d['shape']}  dtype={d['dtype'].__name__}")

dummy = np.zeros(inp["shape"], dtype=inp["dtype"])
interp.set_tensor(inp["index"], dummy)
interp.invoke()
out0 = interp.get_tensor(out[0]["index"])
print("\nOutput[0] shape:", out0.shape)
print("Expected        : (1, 84, 2100)  — 80 COCO classes + 4 box coords")
print("Float input     :", inp["dtype"] == np.float32)
print("\nMODEL VERIFIED!")


## Cell 4 — Download

In [ ]:
import shutil, os
from google.colab import files

os.makedirs("coco_package", exist_ok=True)
shutil.copy(MODEL_PATH, "coco_package/yolov8n_coco_int8.tflite")

shutil.make_archive("coco_obstacle_model","zip","coco_package")
print(f"Zip: {os.path.getsize('coco_obstacle_model.zip')/1024/1024:.1f} MB")
files.download("coco_obstacle_model.zip")
print("Done! Add yolov8n_coco_int8.tflite to your RPi 5 folder.")
